In [11]:
! pip install geopandas
! pip install pulp
! pip install spopt
! pip install coinor-cbc
! pip install matplotlib-scalebar

ERROR: Could not find a version that satisfies the requirement coinor-cbc (from versions: none)
ERROR: No matching distribution found for coinor-cbc


In [12]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-07-04T17:21:18.646778+07:00

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [13]:
import pandas as pd
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.lines as mlines
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
import time
import numpy as np
import pulp
import shapely
import spopt
from spopt.locate import PCenter, simulated_geo_points

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # ignore deprecation warning - GH pysal/spaghetti#649
    import spaghetti

%watermark -w
%watermark -iv

Watermark: 2.4.3

spaghetti          : 1.7.5.post1
pandas             : 2.2.2
matplotlib         : 3.9.0
matplotlib_scalebar: 0.8.1
pulp               : 2.8.0
numpy              : 1.26.4
shapely            : 2.0.4
networkx           : 3.3
geopandas          : 0.14.4
spopt              : 0.6.0



In [14]:
data1 = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/data/csv/network_distance.csv')
# data1.head(6)
data1

,StartPoint,EndPoint,Distance,net_length
0,99,100,10.0,210.0
1,100,101,10.0,0.0
2,101,102,10.0,0.0
3,102,103,10.0,0.0
4,103,104,10.0,0.0
...,...,...,...,...
437,119,115,40.0,0.0
438,119,116,30.0,0.0
439,119,117,20.0,0.0
440,119,118,10.0,0.0


In [15]:
# Lấy các giá trị duy nhất từ cột 'StartPoint'
unique_destinations = set(data1['StartPoint'])

# In ra các giá trị duy nhất dưới dạng DataFrame
unique_destinations_df = pd.DataFrame({'StartPoint': list(unique_destinations)})
print(unique_destinations_df)

    StartPoint
0           99
1          100
2          101
3          102
4          103
5          104
6          105
7          106
8          107
9          108
10         109
11         110
12         111
13         112
14         113
15         114
16         115
17         116
18         117
19         118
20         119


In [16]:
data2 = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/data/csv/Thuyhe_Diemven2_84.csv')
# data2.head(6)
data2

,Id,XX,YY
0,0,612684.883012,1.189163e+06
1,1,612684.883012,1.189173e+06
2,2,612684.883012,1.189183e+06
3,3,612684.883012,1.189193e+06
4,4,612684.883012,1.189203e+06
...,...,...,...
9841,9841,618633.328360,1.197564e+06
9842,9842,618630.751070,1.197554e+06
9843,9843,618628.184968,1.197544e+06
9844,9844,618625.906735,1.197535e+06


In [17]:
# Gộp hai DataFrame dựa trên cột 'Id' và chỉ lấy cột 'name' từ unique_destinations_df
merged_df = pd.merge(unique_destinations_df, data2[['Id', 'XX', 'YY']], left_on='StartPoint', right_on='Id')

# In ra kết quả
print(merged_df)

    StartPoint   Id             XX            YY
0           99   99  613192.959416  1.189768e+06
1          100  100  613202.687189  1.189770e+06
2          101  101  613212.388050  1.189773e+06
3          102  102  613221.483271  1.189777e+06
4          103  103  613230.578492  1.189781e+06
5          104  104  613239.673712  1.189785e+06
6          105  105  613248.768933  1.189789e+06
7          106  106  613257.864154  1.189794e+06
8          107  107  613266.959375  1.189798e+06
9          108  108  613276.054595  1.189802e+06
10         109  109  613285.149816  1.189806e+06
11         110  110  613294.245037  1.189810e+06
12         111  111  613303.340258  1.189814e+06
13         112  112  613312.435479  1.189819e+06
14         113  113  613321.530699  1.189823e+06
15         114  114  613330.625920  1.189827e+06
16         115  115  613339.721141  1.189831e+06
17         116  116  613348.816362  1.189835e+06
18         117  117  613357.911583  1.189839e+06
19         118  118 

In [18]:
# Sắp xếp lại các cột theo thứ tự chieudai, frompoint, topoint
df_sorted = merged_df[['Id', 'StartPoint', 'XX', 'YY']]

print(df_sorted)

     Id  StartPoint             XX            YY
0    99          99  613192.959416  1.189768e+06
1   100         100  613202.687189  1.189770e+06
2   101         101  613212.388050  1.189773e+06
3   102         102  613221.483271  1.189777e+06
4   103         103  613230.578492  1.189781e+06
5   104         104  613239.673712  1.189785e+06
6   105         105  613248.768933  1.189789e+06
7   106         106  613257.864154  1.189794e+06
8   107         107  613266.959375  1.189798e+06
9   108         108  613276.054595  1.189802e+06
10  109         109  613285.149816  1.189806e+06
11  110         110  613294.245037  1.189810e+06
12  111         111  613303.340258  1.189814e+06
13  112         112  613312.435479  1.189819e+06
14  113         113  613321.530699  1.189823e+06
15  114         114  613330.625920  1.189827e+06
16  115         115  613339.721141  1.189831e+06
17  116         116  613348.816362  1.189835e+06
18  117         117  613357.911583  1.189839e+06
19  118         118 

In [19]:
# # Đổi tên các cột
df = df_sorted.rename(columns={"StartPoint": "FacilityPoints"})
print(df)

     Id  FacilityPoints             XX            YY
0    99              99  613192.959416  1.189768e+06
1   100             100  613202.687189  1.189770e+06
2   101             101  613212.388050  1.189773e+06
3   102             102  613221.483271  1.189777e+06
4   103             103  613230.578492  1.189781e+06
5   104             104  613239.673712  1.189785e+06
6   105             105  613248.768933  1.189789e+06
7   106             106  613257.864154  1.189794e+06
8   107             107  613266.959375  1.189798e+06
9   108             108  613276.054595  1.189802e+06
10  109             109  613285.149816  1.189806e+06
11  110             110  613294.245037  1.189810e+06
12  111             111  613303.340258  1.189814e+06
13  112             112  613312.435479  1.189819e+06
14  113             113  613321.530699  1.189823e+06
15  114             114  613330.625920  1.189827e+06
16  115             115  613339.721141  1.189831e+06
17  116             116  613348.816362  1.1898

In [20]:
import os

# Đường dẫn đến thư mục
folder_path = r'C:/Users/ASUS/Downloads/rescue_station_project/data/csv'

# Kiểm tra nếu thư mục không tồn tại, tạo mới
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Tạo đường dẫn đến file CSV trong thư mục
file_path = os.path.join(folder_path, 'facility_points.csv')

# Lưu dataframe thành file CSV
df.to_csv(file_path, index=False)

# Hiển thị kết quả
print("File CSV đã được lưu thành công vào thư mục:", folder_path)

File CSV đã được lưu thành công vào thư mục: C:/Users/ASUS/Downloads/rescue_station_project/data/csv
